In [1]:
import json
from IPython.display import JSON

# Workshop 2: Meeting Scheduler

In this workshop, we will go through a simple example of using Function Calls in ChatCompletion with Azure OpenAI. We will create a personal meeting scheduler to demonstrate this functionality.

## Table of Contents

1. [Introduction](#Introduction)
2. [Creating a Simple Function](#Creating-a-Simple-Function)
3. [Creating JSON Function Tool for Azure OpenAI](#Creating-JSON-Function-Tool-for-Azure-OpenAI)
4. [Using the Function Tool in the ChatCompletion Request](#Using-the-Function-Tool-in-the-ChatCompletion-Request)
5. [Example Usage and Demonstrations](#Example-Usage-and-Demonstrations)

### Introduction

In this workshop, we will cover the following:

1. **Creating a Simple Function**: Define a basic function that simulates scheduling a meeting.
2. **Creating JSON Function Tool for Azure OpenAI**: Create a JSON representation of the function to instruct the OpenAI model on how to call it.
3. **Using the Function Tool in the ChatCompletion Request**: Integrate the function tool into the ChatCompletion request to handle user requests for scheduling meetings.
4. **Example Usage and Demonstrations**: Provide practical examples and demonstrations of the functionality, including how to handle responses and execute the function calls.

Let's get started by importing the necessary libraries and setting up the Azure OpenAI client.

We will use the imports, functions mentioned in workshop_1 

In [2]:
from openai import AzureOpenAI

In [3]:
client = AzureOpenAI(
    azure_endpoint="https://testopenai4.openai.azure.com/",
    api_key="19f61bb2a418484dbe809cb720c2527b",
    api_version="2024-02-15-preview"
)

## 1. Creating a Function

We will define a simple function that simulates scheduling a meeting in a calendar. This function will take a title, date and time, and a list of invitees as input, and return a confirmation message.

In [11]:
def schedule_meeting(title: str, date_and_time: str, invitees: list['str']):
    """This function will schedule a new meeting"""
    return f"A new meeting '{title}' at: {date_and_time}, with invitees: {invitees} created sucessefully" 

In [12]:
schedule_meeting("meeting", "02/03/2024", ["John Week", "Ajit Moe", "Dor Getter"])

"A new meeting 'meeting' at: 02/03/2024, with invitees: ['John Week', 'Ajit Moe', 'Dor Getter'] created sucessefully"

## 2. Creating a Function Call JSON

We will create a JSON representation of our meeting scheduling function. This JSON will be used to instruct the OpenAI model on how to call the function.

### Overall Structure

A function call JSON consists of:
- **type**: Specifies that this is a function call definition.
- **function**: An object that contains:
  - **name**: The name of the function.
  - **description**: A brief description of what the function does.
  - **parameters**: An object defining the parameters the function accepts, including their types and descriptions.

### Function Call Parameter Types

When defining parameters for function calls, you can use the following types:

1. **String (`string`)**: Textual data.
2. **Number (`number`)**: Numeric data (integers and floating-point).
3. **Integer (`integer`)**: Integer values.
4. **Boolean (`boolean`)**: True or false.
5. **Array (`array`)**: List of items.
6. **Object (`object`)**: Complex structure with properties.

<details>
  <summary>Click to expand full example</summary>

  ```json
  {
    "type": "object",
    "properties": {
      "title": { "type": "string" },
      "date_and_time": { "type": "string" },
      "invitees": { "type": "array", "items": { "type": "string" } }
    }
  }
</details>
"""
```

In [13]:
schedule_meeting_tool = {
    "type": "function",
    "function": {
        "name": "schedule_meeting",
        "description": "Schedules a new meeting with a specified title, date and time, and list of invitees.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {
                    "type": "string",
                    "description": "The title of the meeting."
                },
                "date_and_time": {
                    "type": "string",
                    "description": "The date and time of the meeting in a specified format."
                },
                "invitees": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "A list of invitees' names or email addresses."
                }
            },
            "required": ["title", "date_and_time", "invitees"],
        },
    }
}

## 3. Using the function call in ChatCompletion:

In [14]:
system_message = """
You are a helpful assistant capable of scheduling meetings.
when a user asks to schedule a meeting make sure that the user specifies the following entities:
- **title**: Meeting title (string)
- **date_and_time**: Date and time (string)
- **invitees**: List of invitees' names (can be only one name)
Instructions:
When a user requests to schedule a meeting, ensure you have all the required entities needed to call the scheduling tool. 
If any information is missing, explicitly ask for the missing fields. if user provided only one name than dont ask him for more invitees.
"""
input_text = "schedule a meeting with Dor Avihu, Shmuel Shalom and Lyri Anderson at september 8 2024 with title 'Microsoft Meeting'"
tools = [schedule_meeting_tool]

In [15]:
response = client.chat.completions.create(
    model = "gpt-4-32k",
    messages = [{"role": "system", "content": system_message},
                {"role": "user", "content":input_text}],
    temperature= 0,
    tools=tools, 
    tool_choice="auto",
)
JSON(json.loads(response.model_dump_json()))

<IPython.core.display.JSON object>

In [16]:
print(response.choices[0].message.role, ":",response.choices[0].message.content)
print("----------")
print(response.choices[0].message.tool_calls[0].function.name, ": with args:",response.choices[0].message.tool_calls[0].function.arguments)

assistant : None
----------
schedule_meeting : with args: {
  "title": "Microsoft Meeting",
  "date_and_time": "september 8 2024",
  "invitees": ["Dor Avihu", "Shmuel Shalom", "Lyri Anderson"]
}


In [17]:
tool_calls = response.choices[0].message.tool_calls[0]
function_name = tool_calls.function.name
function_arguments = json.loads(tool_calls.function.arguments)
function_arguments.get('title')

'Microsoft Meeting'

we get back the response and now we will pull out the tool calls:

In [18]:
tool_calls = response.choices[0].message.tool_calls[0]

if tool_calls:
    # To be able to use our tools we will need to create a mapping between the function name and the actual function object:
    available_functions = {
    "schedule_meeting": schedule_meeting
    }
    function_name = tool_calls.function.name
    function_arguments = json.loads(tool_calls.function.arguments)
    function_to_call = available_functions[function_name]
    function_response = function_to_call(
        title= function_arguments.get('title'),
        date_and_time= function_arguments.get("date_and_time"),
        invitees= function_arguments.get("invitees"),
    )
function_response

"A new meeting 'Microsoft Meeting' at: september 8 2024, with invitees: ['Dor Avihu', 'Shmuel Shalom', 'Lyri Anderson'] created sucessefully"